# First Exercise

I calculated the posterior probability with this way. First of all we have to consider that a false positive occurs when all the hashing functions returns a value that is set to true into the bloom filter and this value is **k**, second, all the events H_i = {the i-th hashing function returns a value that is set to true into the BloomFilter} are independent. So the P(A|B) = P(A and B)/P(B) = P(A) = P(H_1 and H_2) = P(H_1) * P(H_2) =  Ber(p) where: A = {false positive}, B = {the password isn't contained into the first set} = omega **p = |variables set to true| / #lenght of the bloomFilter array**

In [1]:
import time
import numpy as np

def hashFun1(s, bloomLen):
    res = 1
    for i in range(len(s)):
        res *= int(ord(s[i]) / (i+2))
    return res % bloomLen

def hashFun2(s, bloomLen):
    res = 1
    for i in range(int(len(s)/2)):
        res *= int(ord(s[i]) / (i+1))
    return res % bloomLen

def hashFun3(s, bloomLen):
    res = 2166136261
    for i in range(len(s)):
        res ^= ord(s[i]); res *= 16777619
    return res % bloomLen

def hashFun4(s, bloomLen):
    a = b = c = 0
    for i in range(len(s)//5):
        if i <= 1:
            a += ord(s[i])<<i + ord(s[i+1])<<(i+1)
            b += ord(s[i+2])<<i + ord(s[i+3])<<(i+1)
            c += ord(s[i+4])<<i
        else:
            a += ord(s[i])<<(i-1) + ord(s[i+1])<<i
            b += ord(s[i+2])<<(i-1) + ord(s[i+3])<<i
            c += ord(s[i+4])<<(i-1)
    return (a+b-c) % bloomLen

def hashFun5(s, bloomLen):
    a = b = 0
    for i in range(len(s)//4):
        a += ord(s[i])<<(i) + ord(s[i+1])<<(i+1)
        b += ord(s[i+2])<<(i) + ord(s[i+3])<<(i+1)
    return (a+b) % bloomLen

def initBloomFilter(bloomLen, path):
    bloomFil = np.array([False for i in range(bloomLen)])
    fPass1 = open(path, "r", encoding="utf-8")
    for i in range(100000000):
        password = fPass1.readline()[:20]

        # changes the values according the values returned by the hashing functions
        bloomFil[hashFun1(password, bloomLen)] = True; bloomFil[hashFun2(password, bloomLen)] = True
        bloomFil[hashFun3(password, bloomLen)] = True; bloomFil[hashFun4(password, bloomLen)] = True
        bloomFil[hashFun5(password, bloomLen)] = True
    fPass1.close()
    return bloomFil
    

def passCheck(path, bloomLen, bloomFil):
    n = 0
    fPass2 = open(path, "r", encoding="utf-8")
    for i in range(3900000):
        password = fPass2.readline()[:20]
        h1 = hashFun1(password, bloomLen); h2 = hashFun2(password, bloomLen)
        h3 = hashFun3(password, bloomLen); h4 = hashFun4(password, bloomLen); h5 = hashFun5(password, bloomLen)
        if bloomFil[h1] == True and bloomFil[h2] == True and bloomFil[h3] == True and bloomFil[h4] == True and bloomFil[h5] == True:
            n += 1
    fPass2.close()
    return n

# initialisation of variables, I choose 725000000 as the lenght of the bloom filter because to have a prior probability of 
# about 3% i have to do 5 hashing functions and a vector of that size
k = 5; bloomLen = 725000000

start = time.time()
# inizialises the bloomFilter array of bit values
blFilter = initBloomFilter(bloomLen, "C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM4\\passwords1.txt")

# check all passwords2 presence in the filter
n = passCheck("C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM4\\passwords2.txt", bloomLen, blFilter)

# calculating the prior and posterior probability of false positive
pPr = ((1 - (1/2.71) ** ((k * 100000000) / bloomLen)) ** k)
numTrue = 0
for i in range(len(blFilter)):
    if blFilter[i] == True:
        numTrue += 1
pPos = (numTrue/bloomLen) ** k

end = time.time()

print('Number of hash function used: ', k)
print('Number of duplicates detected: ', n)
print('Probability of false positives previous the creation of the bloom filter: ' + str(round(pPr * 100, 7)) + "%")
print('Probability of false positives after the creation of the bloom filter: ' + str(round(pPos * 100, 7)) + "%")
print('Execution time: ', end-start)

Number of hash function used:  5
Number of duplicates detected:  1561326
Probability of false positives previous the creation of the bloom filter: 3.0382899%
Probability of false positives after the creation of the bloom filter: 0.1086364%
Execution time:  2908.527927875519
